In [ ]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import HTML

from bqplot import Figure, Scatter, Axis, LinearScale

In [ ]:
EXPLANATION = """\
<div class="app-sidebar">
<p><em>Compare different development indicators.</em><p>

<p>Select what indicators to plot in the dropdowns, and use the slider
to sub-select a fraction of years to include in the plot.</p>

<p>Data and idea copied from the <a href="https://dash.plot.ly/getting-started-part-2">
Plotly Dash documentation</a>.</p>

<p>This example demonstrates combining <a href="https://bqplot.github.io/bqplot">
bqplot</a> with Jupyter widgets.</p>
</div>
"""

In [ ]:
class App:

    def __init__(self, df):
        self._df = df
        available_indicators = self._df['Indicator Name'].unique()
        self._x_dropdown = self._create_indicator_dropdown(available_indicators, 0)
        self._y_dropdown = self._create_indicator_dropdown(available_indicators, 1)

        x_scale = LinearScale()
        y_scale = LinearScale()

        self._x_axis = Axis(scale=x_scale, label="X")
        self._y_axis = Axis(scale=y_scale, orientation="vertical", label="Y")

        self._scatter = Scatter(
            x=[], y=[], scales={"x": x_scale, "y": y_scale}
        )

        self._figure = Figure(marks=[self._scatter], axes=[self._x_axis, self._y_axis], layout=dict(width="99%"), animation_duration=1000)

        self._year_slider, year_slider_box = self._create_year_slider(
            min(df['Year']), max(df['Year'])
        )
        _app_container = widgets.VBox([
            widgets.HBox([self._x_dropdown, self._y_dropdown]),
            self._figure,
            year_slider_box
        ], layout=widgets.Layout(align_items='center', flex='3 0 auto'))
        self.container = widgets.VBox([
            widgets.HTML(
                (
                    '<h1>Development indicators. A Voici dashboard, running entirely in your browser!</h1>'
                    '<h2 class="app-subtitle"><a href="https://github.com/pbugnion/voila-gallery/blob/master/country-indicators/index.ipynb">Link to code</a></h2>'
                ),
                layout=widgets.Layout(margin='0 0 5em 0')
            ),
            widgets.HBox([
                _app_container,
                widgets.HTML(EXPLANATION, layout=widgets.Layout(margin='0 0 0 2em'))
            ])
        ], layout=widgets.Layout(flex='1 1 auto', margin='0 auto 0 auto', max_width='1024px'))
        self._update_app()

    @classmethod
    def from_csv(cls, path):
        df = pd.read_csv(path)
        return cls(df)

    def _create_indicator_dropdown(self, indicators, initial_index):
        dropdown = widgets.Dropdown(options=indicators, value=indicators[initial_index])
        dropdown.observe(self._on_change, names=['value'])
        return dropdown

    def _create_year_slider(self, min_year, max_year):
        year_slider_label = widgets.Label('Year range: ')
        year_slider = widgets.IntRangeSlider(
            min=min_year, max=max_year,
            layout=widgets.Layout(width='500px'),
            continuous_update=False
        )
        year_slider.observe(self._on_change, names=['value'])
        year_slider_box = widgets.HBox([year_slider_label, year_slider])
        return year_slider, year_slider_box

    def _on_change(self, _):
        self._update_app()

    def _update_app(self):
        x_indicator = self._x_dropdown.value
        y_indicator = self._y_dropdown.value
        year_range = self._year_slider.value

        with self._scatter.hold_sync():
            df = self._df[self._df['Year'].between(*year_range)].dropna()
            x = df[df['Indicator Name'] == x_indicator]['Value']
            y = df[df['Indicator Name'] == y_indicator]['Value']

            self._x_axis.label = x_indicator
            self._y_axis.label = y_indicator

            self._scatter.default_opacities = [0.2]

            self._scatter.x = x
            self._scatter.y = y

In [ ]:
app = App.from_csv("./indicators.csv")

app.container

## Choosing Between Xeus and Pyodide for JupyterLite

JupyterLite supports two main Python backends: **Pyodide** and **xeus-python**. Each has its own strengths, and choosing between them depends on your project's needs.

- **Pyodide**: 
  - Allows users to dynamically add Python packages in the browser, providing flexibility if they need additional libraries.
  - When users load your JupyterLite deployment for the first time, the required packages are installed via `pip` in the browser, which can lead to slightly longer initial load times.
  
- **Xeus-Python**:
  - Packages are pre-installed, meaning users cannot `pip install` additional packages during a session. However, this setup provides a faster, more streamlined experience since dependencies are baked into the deployment.
  - Seems to support a broader range of packages compared to Pyodide, offering better compatibility for complex projects.


## Creating a local development environment for Jupyterlite.


### Step 1: Install Micromamba

To manage the environment and dependencies, we’ll use **Micromamba**, a lightweight package manager compatible with Conda’s ecosystem.

1. **Install Micromamba** 

(via Homebrew on macOS):
```bash
brew install micromamba
```

via apt on linux/WSL
```bash
sudo apt install micromamba
```

2. **Initialise Micromamba** for your shell (optional): This step sets up Micromamba for your shell, allowing you to use it seamlessly in your terminal.

```bash
micromamba shell init -s bash
micromamba shell init -s zsh
```

3. **Source the Shell Configuration File:** After initializing, source your relevant shell configuration file to load Micromamba.

```bash
source ~/.zshrc  # or source ~/.bashrc
```

### Step 2: Set Up the Build Environment

1. **Create the Build Environment:** With Micromamba, create a build environment defined in your build-environment.yml file. This setup will include all required dependencies for JupyterLite and the xeus-python kernel.

```bash
micromamba create -f build-environment.yml
```

2. **Activate the Environment:** Activate the build environment to install and run JupyterLite.

```bash
micromamba activate build-env
```

### Step 3: Build and Run a Local Version of JupyterLite

1. **Initialize JupyterLite:** Run the following command to initialize JupyterLite in your project directory. This command creates the initial setup files.

```bash
jupyter lite init
```

2. **Build and Serve JupyterLite Locally:** Now, build and serve JupyterLite locally

```bash
jupyter lite serve --contents content  # the content folder contains the notebooks that will be served.
```